In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Safari()

driver.get("https://www.agoda.com/sindhorn-kempinski-bangkok/hotel/bangkok-th.html?cid=1844104")
assert "Agoda" in driver.title

elem = driver.find_element(By.CLASS_NAME, "room-gallery-link")
elem.click
# assert "No results found." not in driver.page_source
elem.location_once_scrolled_into_view

AssertionError: 

In [1]:
import requests
import json
import time

Search to get property ids.
Then use those ids to get each propety details.


## Extract

### search
- Get property id to input GetSecondaryData

In [119]:
'''
9395    = BKK
17019   = Hua Hin
17198   = Samui
'''

def search_props(city_id, check_in_date):
    '''
    "2023-05-06"
    '''
    
    # Setup query
    with open('./search.json', 'rb') as f:
        search_query = json.load(f)
    
    search_query['variables']['CitySearchRequest']['cityId'] = city_id
    search_query['variables']['CitySearchRequest']['searchRequest']['searchCriteria']['bookingDate'] = f"{check_in_date}T08:46:25.999Z"
    search_query['variables']['CitySearchRequest']['searchRequest']['searchCriteria']['checkInDate'] = f"2023-05-05T17:00:00.000Z"
    search_query['variables']['CitySearchRequest']['searchRequest']['searchCriteria']['localCheckInDate'] = check_in_date
    search_query['variables']['CitySearchRequest']['searchRequest']['page']['pageSize'] = 100
    search_query['variables']['CitySearchRequest']['searchRequest']['page']['pageNumber'] = 1

    url = 'https://www.agoda.com/graphql/search'

    headers = {
            'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
            'Content-Type': 'application/json',
            'AG-LANGUAGE-LOCALE': 'en-gb'
        }


    # Output
    r = requests.post(url=url, headers=headers, json=search_query)
    search_result = r.json()

    
    # # Get totalFilteredHotels from search_result
    # totalFilteredHotels = search_result['data']['citySearch']['searchResult']['searchInfo']['totalFilteredHotels']
    properties = search_result['data']['citySearch']['properties']
    # # Get max page
    # max_page = (totalFilteredHotels // 100) + 1
    # print(totalFilteredHotels)
    
    
    # Loop page until reach maximum number of props
    # for i in range(2, max_page+1):
    #     search_query['variables']['CitySearchRequest']['searchRequest']['page']['pageNumber'] = i
    #     r = requests.post(url=url, headers=headers, json=search_query)
    #     search_out = r.json()
    #     properties = properties + search_out['data']['citySearch']['properties']
    #     # print(len(properties))
    
    # Iterate until recieve no more property
    while True:
        
        search_query['variables']['CitySearchRequest']['searchRequest']['page']['pageNumber'] += 1
        r = requests.post(url=url, headers=headers, json=search_query)
        search_out = r.json()
        search_result['data']['citySearch']['properties'] += search_out['data']['citySearch']['properties']
        time.sleep(0.2)
        print(len(search_out['data']['citySearch']['properties']))
        if len(search_out['data']['citySearch']['properties']) == 0:
            break

    # search_result['data']['citySearch']['properties'] = properties
    return search_result

In [120]:
search_result = search_props(city_id=9395, check_in_date= "2023-05-06")

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
93
85
78
74
74
80
78
48
19
0


In [101]:
# Save to file
with open('./search_result.json', 'w') as f:
    json.dump(search_result, f)

In [ ]:
props = search_result['data']['citySearch']['properties']
for prop in props:
    id = prop['propertyId']
    name = prop['content']['localeName']

### property

- Run once to initiate property database. In property page, showing property info.

In [2]:
# assert 1624710 == 'SALA Samui Chaweng Beach Resort'
# 267449 == 'Conrad Koh Samui (SHA Plus+)'

def extract_prop_info(property_id:list, prop_quary:dict) -> dict:    
    url = 'https://www.agoda.com/graphql/search'
    
    prop_quary['variables']['PropertyDetailsRequest']['propertyIds'] = property_id

    headers = {
            'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
            'Content-Type': 'application/json',
            'AG-LANGUAGE-LOCALE': 'en-gb'
        }

    r = requests.post(url=url, headers=headers, json=prop_quary)
    
    # Assert request id with response id
    
    return r.json()

In [49]:
with open('./property_query.json', 'rb') as f:
    quary_json = json.load(f)
    
data_out = extract_prop_info([1624710, 267449], quary_json=quary_json)

In [57]:
def save_prop_info(data:json):
    
    props = data["data"]["propertyDetailsSearch"]['propertyDetails']

    for prop in props:
        id = prop['propertyId']
        name = prop['contentDetail']['contentSummary']['displayName']
        
        print(f'Writing {id}, {name}')

        with open(f'./{id}_{name}_info.json', 'w') as f:
            json.dump(data, f)
        
        print(f'Writing {id}, {name} ..done')

In [58]:
save_prop_info(data=data_out)

Writing 1624710, SALA Samui Chaweng Beach Resort  (SHA Extra Plus)
Writing 1624710, SALA Samui Chaweng Beach Resort  (SHA Extra Plus) ..done
Writing 267449, Conrad Koh Samui (SHA Plus+)
Writing 267449, Conrad Koh Samui (SHA Plus+) ..done


### GetSecondaryData

- In property page showing prices.

In [35]:
def get_hotel_info(check_in_date:str, hotel_id:str, session_id:str) -> json:
    
    '''
    A function that return hotel
    params:
        check_in_date = check in date in string format %Y%m%d
    '''

    url = f'https://www.agoda.com/api/cronos/property/BelowFoldParams/GetSecondaryData?finalPriceView=1&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&checkIn={check_in_date}&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1&showReviewSubmissionEntry=false&currencyCode=THB&isFreeOccSearch=false&isCityHaveAsq=true&los=1&hotel_id={hotel_id}&all=false&price_view=1&sessionid={session_id}'

    headers = {
            "Accept-Language": "en-GB, en-US;q=0.9, en;q=0.8",
            "Content-Type": "application/json; charset=UTF-8",
            "CR-Currency-Code": "THB",
        }

    # x-raw-url: /en-gb/sindhorn-kempinski-bangkok  /hotel/bangkok-th.html?     finalPriceView=1&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&checkIn=2023-06-7&  isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1     &showReviewSubmissionEntry=false&currencyCode=THB&isFreeOccSearch=false&isCityHaveAsq=true&los=1&searchrequestid=1d454872-6ad5-43fe-8176-2f746d4ad9ab
    # x-raw-url: /en-gb/kimpton-maa-lai-bangkok     /hotel/bangkok-th.html?     finalPriceView=1&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&checkin=2023-06-16& isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=1      &showReviewSubmissionEntry=false&currencyCode=THB&isFreeOccSearch=false&isCityHaveAsq=true&los=1&searchrequestid=353aad33-3ffb-4142-b4c6-129a17da3ae0
    # URL: https://www.agoda.com/api/cronos/property/BelowFoldParams/GetSecondaryData?finalPriceView=1&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&     checkin=2023-06-16  isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=1   &showReviewSubmissionEntry=false&currencyCode=THB&isFreeOccSearch=false&isCityHaveAsq=true&los=1    &searchrequestid=353aad33-3ffb-4142-b4c6-129a17da3ae0       &hotel_id=10306365  &all=false&price_view=1&sessionid=ut0pv141kmrgktolqzgc4iaf&pagetypeid=7
    # URL: https://www.agoda.com/api/cronos/property/BelowFoldParams/GetSecondaryData?finalPriceView=1&isShowMobileAppPrice=false&cid=-1&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&     checkIn=2023-06-7&  isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=-1  &showReviewSubmissionEntry=false&currencyCode=THB&isFreeOccSearch=false&isCityHaveAsq=true&los=1    &searchrequestid=1d454872-6ad5-43fe-8176-2f746d4ad9ab       &hotel_id=11027646  &all=false&price_view=1&sessionid=ut0pv141kmrgktolqzgc4iaf&pagetypeid=7

    return requests.get(url=url, headers=headers)

In [36]:
check_in_date = '2023-06-7'
hotel_id = '12505308'
# 11027646
session_id = 'ut0pv141kmrgktolqzgc4iaf'

r = get_hotel_info(check_in_date, hotel_id, session_id)
r.json()

{'protocol': 'https:',
 'translations': {'hostInfo': {'viewHostProfile': 'View host profile',
   'verifiedHost': 'Verified Host',
   'topHost': 'Top Host',
   'joinedIn': 'Joined in {0}',
   'fastResponse': 'Fast response',
   'readMore': 'Read more',
   'contactHost': 'Contact host'},
  'stealTheDealBanner': {'agpText': 'Recommended for you by our prestigious Agoda Preferred Program',
   'nonAgpText': 'Recommended for your stay in {0}',
   'viewPropertyText': 'View property'},
  'hotelGuestPolicies': {'guestPoliciesChildAndExtrabedTitle': 'Children and extra beds',
   'guestAge': "Guest's age",
   'extraBed': 'Extra bed',
   'guestPoliciesOtherTitle': 'Others',
   'guestPoliciesAskProperty': 'Extra beds, if available, are dependent on the room you choose. Please ask the property for more details.',
   'askTheProperty': 'ASK THE PROPERTY'},
  'hotelFaq': {'title': 'Frequently asked questions',
   'showMoreQuestion': 'Show more questions',
   'viewMap': 'View Map',
   'exploreNeighborho

In [ ]:
# or get from = data-hotelid="10640"
# in class="hotel-list-container"

In [ ]:
with open('./search_result.json', 'r') as f:
    json = json.load(f)
    
props = json['data']['citySearch']
